<a href="https://colab.research.google.com/github/Rithikareddy-02/AI/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import Libraries

In [ ]:
import numpy as np
import math as mt
import random
import pandas as pd

Import Data

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/DATA_AI/Ground_water_data.csv')
data.head()

,X,Y
0,7.6,157
1,7.1,174
2,8.2,175
3,7.5,188
4,7.4,171


Outlier Detection

In [ ]:
outliers = []
def detect_outliers_zscore(data):
    thres = 3
    mean = np.mean(data)
    std = np.std(data)
    # print(mean, std)
    for i in data:
        z_score = (i-mean)/std
        if (np.abs(z_score) > thres):
            outliers.append(i)
    return outliers# Driver code

In [ ]:
sample_outliers_A = detect_outliers_zscore(data['X'])
sample_outliers_B = detect_outliers_zscore(data['Y'])
print("Outliers in A column from Z-scores method: ", sample_outliers_A)
print("Outliers in B column from Z-scores method: ", sample_outliers_B)

Outliers in A column from Z-scores method:  []
Outliers in B column from Z-scores method:  []


Outliers Treatment

In [ ]:
from scipy.stats import zscore
zA = np.abs(zscore(data['X']))
zB = np.abs(zscore(data['Y']))
data['ZscoreX'] = zA
data['ZscoreY'] = zB
outliers_A=len(data[data['ZscoreX']>3])
outliers_B=len(data[data['ZscoreY']>3])
data_z = data[data['ZscoreX']<3]
data=data_z.drop(['ZscoreX', 'ZscoreY'], axis=1)

Missing Values Treatment

In [ ]:
print(data.isnull().sum())

X    0
Y    0
dtype: int64


In [ ]:
m1=data['X'].mean()
m2=data['Y'].mean()
data['X'].fillna(value=m1, inplace=True)
data['Y'].fillna(value=m2, inplace=True)

Normalisation

In [ ]:
from sklearn.preprocessing import MinMaxScaler
X1 = np.array(data['X']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(X1)
X_scaled = scaler.transform(X1)
data['X'] = X_scaled.reshape(1,-1)[0]

In [ ]:
Y1 = np.array(data['Y']).reshape(-1,1)
scaler = MinMaxScaler()
scaler.fit(Y1)
Y_scaled = scaler.transform(Y1)
data['Y'] = Y_scaled.reshape(1,-1)[0]

Min and Max Values

In [ ]:
x_min = data['X'].min()
x_max = data['X'].max()
y_min = data['Y'].min()
y_max = data['Y'].max()

Data Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.X, data.Y, test_size = 0.20, random_state = 10)

Initilization

In [ ]:
m1=random.uniform(-10,10)
m2=random.uniform(-10,10)
c=random.uniform(-10,10)
eta=0.01
max_iter=100000

Gradient Calculation

In [ ]:
def gradient(m1,m2,c,x,y):
  m1_deriv=-1*(y-m1*x*x-m2*x-c)*x*x
  m2_deriv=-1*(y-m1*x*x-m2*x-c)*x
  c_deriv=-1*(y-m1*x*x-m2*x-c)
  return m1_deriv,m2_deriv,c_deriv

Training the Model

In [ ]:
for iter in range(max_iter):
  for sample in range(x_train.shape[0]):
    del_m1,del_m2,del_c=gradient(m1, m2, c, x_train.iloc[sample], y_train.iloc[sample])
    m1=m1-eta*del_m1
    m2=m2-eta*del_m2
    c=c-eta*del_c
print('model parameter m1, m2 and c',m1,m2,c)

model parameter m1, m2 and c -0.21080839453697003 -0.13545338659968997 0.5663243740172735


Model Prediction on Training Data

In [ ]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m1 * i*i)+(m2 * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

Model Prediction on Testing Data

In [ ]:
y_pred_test = []
for i in x_test:
  y_p_te = (m1 * i*i)+(m2 * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

Error Metrics

In [ ]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
rmse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', rmse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.2134300260886146
Mean square error 0.04555237603618671
Mean absolute error 0.18685609819380386


In [ ]:
#Testing Accuracies
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse)
mse = mt.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse)  
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Mean square error 0.0779203240291803
Root mean square error 0.27914212156029106
Mean absolute error 0.2406795981608483


Deployment

In [ ]:
X=float(input('Enter PH value of well water'))
print('PH value is ',X)

x_nor = (X - x_min)/(x_max - x_min)
y_nor = m1*x_nor*x_nor + m2*x_nor + c

Y = y_nor*(y_max - y_min) + y_min 
print('Bi-carbonates of well water ',Y)

Enter PH value of well water65
PH value is  65.0
Bi-carbonates of well water  -898.9036126736609
